In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random as rn
from my_utils import Workout_dataset
from my_model import make_CNN_RNN_model

import os

os.environ['PYTHONHASHSEED'] = str(42)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.random.set_seed(42)
np.random.seed(42)
rn.seed(42)

In [2]:
train_dir = './data/train'
eval_dir = './data/train'
eval_label_dir = './data/data_y_train.csv'
label_dir = './data/data_y_train.csv'

def scheduler(epoch, lr):
    if (epoch>20) and (lr > 0.00001):
        lr = lr*0.9
        return lr
    else:
        return lr

train_y = pd.read_csv('./data/data_y_train.csv')  # label load
label_dict = dict()
for label, label_desc in zip(train_y.label, train_y.label_desc):
    label_dict[label] = label_desc
# 'Squat (kettlebell / goblet)'에서 [/]를 [,]으로 변경
label_dict[45] = 'Squat (kettlebell , goblet)'

In [3]:
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
BATCH_SIZE = 64

train_loader = Workout_dataset(
    train_dir, label_dir, mode='Train',
    fold=0, batch_size=BATCH_SIZE, augment=True, shuffle=True)

valid_loader = Workout_dataset(
    train_dir, label_dir, mode='Valid',
    fold=0, batch_size=16, shuffle=True)


In [4]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    'balanced',
    np.unique(train_y.label),
    train_y.label)

class_weight_dict = dict(zip(
    list(range(61)),
    class_weights+1
    ))

In [5]:
model = make_CNN_RNN_model(
    leakyrelu_alpha = 0.2,
    input_kernels = 10,
    input_kernel_width = 3,
    res_kernels = 60,
    res_kernel_width = 3,
    lr = 0.001,
    res_num = 3)

model.summary()

__________________________________________________________________________________
conv1d (Conv1D)                 (None, 300, 10)      910         concatenate[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 300, 10)      910         concatenate_1[0][0]              
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 300, 10)      730         concatenate_3[0][0]              
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 300, 10)      1090        concatenate_4[0][0]              
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 300, 10)      1090        concatenate_2[0][0]              
__________________________

In [6]:
checkpoint_filepath = "./save/cnn_gru_best.hdf5"

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',min_delta=0.0001,
    patience=20,verbose=1)

history = model.fit_generator(
    generator=train_loader,
    validation_data=valid_loader,
    epochs=2000,
    callbacks=[save_best,early_stop,lr_scheduler],
    class_weight=class_weight_dict)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/2000
34/34 [==============================] - 28s 240ms/step - loss: 224.3997 - accuracy: 0.3525 - val_loss: 145.6070 - val_accuracy: 0.4453

Epoch 00001: val_loss improved from inf to 145.60704, saving model to ./save/cnn_gru_best.hdf5
Epoch 2/2000
34/34 [==============================] - 5s 152ms/step - loss: 107.7432 - accuracy: 0.5230 - val_loss: 73.6141 - val_accuracy: 0.4531

Epoch 00002: val_loss improved from 145.60704 to 73.61409, saving model to ./save/cnn_gru_best.hdf5
Epoch 3/2000
34/34 [==============================] - 5s 151ms/step - loss: 56.48

In [8]:
test_dir = './data/test'
test_label_dir = './data/data_y_test.csv'

In [9]:
test_loader = Workout_dataset(
        test_dir, test_label_dir, mode='Test', batch_size=625, shuffle=False)

In [10]:
model.load_weights(checkpoint_filepath)
model.evaluate_generator(generator=test_loader,verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
1/1 [==============================] - 1s 990ms/step - loss: 0.7637 - accuracy: 0.8192


[0.7636738419532776, 0.8191999793052673]